# COGS 118A - Final Project

#### For beauty and cleanness, we do not include any code in the final report. 

#### Go to [CompleteProject](https://nbviewer.org/github/COGS118A/Group021-Wi23/blob/main/FinalProjectCompleteVersion.ipynb) for all complete code.

Press CTRL or CMD and click the link to open a new tab, so you don't have to go back to this reoprt.

# What's the Music?

## Group members

- Jerry Wang, A16149329
- Darius Azure, A15786330
- Miko Brown, A16237566

# Abstract
Music has been created over thousands of years, people considered music as an expression of beauty, aesthetics, and art. However, it is rare to see people apply modern technology to analyze music. We want to explore the secret of music with a more scientific and computational way.

The goal of our project is to classify song genre based on a variety of features describing the music. Our project tackles the problem of classifying song genre based on many variables including artist, song name, popularity, acousticness, danceability, energy, instrumentalness, key, tune, etc.

The data contains entries with each of these features and their musical genre. We will create pipeline models to perform multi-classification on genre in the dataset. By comparing performance and metrics from different models, we will pick the highest accuracy algorithm by comparing the metrics from each one.

In this project, we will select models pf Logstic Regression(naive tryout),Random Forest, Boosting, and Neural Network, and perform all of them in comparion.

So, you will see three different models here: Random Forest led by Miko, Boosting led by Darius, Neural Network led by Jerry.

In this project, we will use $f_1$ score to determine which model has the best performance. We will investigate precision and recall individually as well, but $f_1$ score will be the determining metric.

# Background
In the history of music, there was no such complex system to define music genre in earlier times. It was more simple to categorize music according to its period of time<a name="wiki1"></a>[<sup>[1]</sup>](#wikinote1). After the 20th century, creative musicians brought different format of music to the stage so that people start to categorize the music into genres. The music genre becomes more complicated as the technology develops. Nowadays, when facing countless music work, we cannot categorize a genre from its time. Instead, we may need to determine the music category based on its features, attributes, and listening response. 

A music genre or subgenre may be defined by the musical techniques, the cultural context, and the content and spirit of the themes<a name="wiki2"></a>[<sup>[2]</sup>](#wikinote2). This article describes how many people subjectively view genre of music. This is useful for human-being, but difficult to classify large datasets of songs without listening to and understanding each song in depth. We can find that music genre itself is not easy to identified by individual factors from listening since people may have subjective thoughts. We tend to believe that there exists some more objective methods in machine learning to categorize music genre with computer algorithms.

Additionally, music genre is a dynamic concept. Due to the natural evolution of music, it may appear differently in differnt periods of time. Therefore, a listening response might be outdated if a genre is gradually disappearing or shifting to different formats<a name="newyorker"></a>[<sup>[3]</sup>](#newyorkernote). In contradiction, data implementation will not be affected by such evolution since data is measure objectively during that time period and measures a fixed in-time genre rather than a dynamic genre shifted from earlier times. For instance, the general keys of Jazz music in 1900s are different with Jazz music in 2020, but the computer will filter out the era's influence on keys.

# Problem Statement
Given different attributes of one song, what is its genre?

The problem is quantifiable, we can represent the problem as:

- Denote song entries $x_1$ to $x_n$ as X
- Denote song genre $y_1$ to $y_n$ as y
- Given $x_1$ to $x_n$, each $\vec x_i$ $\in \mathbb{R}^d$ with features $f_1$ through $f_d$. What is $\bar y_i$?

ML-relevant potential solution: Give $x_1$ to $x_n$, by applying either LogisticRegression, RandomForest, Boosting, and Neural Network, the model produces the prediction of its most likely class $y_i$.

The problem is measurable since we can evaluate the quantitative prediction performance with precision, recall, and f1 score.

The problem is replicable since our selected algorithms are generally deterministic, once the data of a song is obtained, the problem can be reproduced by applying the same algorithms.

# Data
- source: interest in music, general search online for datasets; [author](https://www.kaggle.com/vicsuperman) is from Wellington, New Zealand; the origin source of the data is unknown, we only grab it from Kaggle platform

- reference: [Kaggle](https://www.kaggle.com/datasets/vicsuperman/prediction-of-music-genre), dataset search under classification

- description dataset: 17 of variables, 49944 of observations

- an observation consists of: instance_id, artist_name, track_name, popularity, acousticness, danceability,  duration_ms, energy, instrumentalness, key, liveness, loudness, mode, speechiness, tempo, obtained_date, valence, music_gengre

- Actually we are not sure what should be considered as critical variables since any of the attributes of a song can make huge differernce to the song. Generally, the variables are presented in numerical values and categorical values and contains missing data.

- cleaning of extreme numerical values, unmatched type, different format of categorical values

- data missingness: if the data contains missing value, we need to first examine its type: Missing Completely At Random (MCAR), Missing At Random (MAR), or Missing Not At Random (MNAR). Then, we need to apply the strategy from one of the following: 1. ignore missingness 2. fill with special value 3.fill with mean/median 4.fill with nearest data 5.fill with our model prediction. This decision will be further determined in the data part if we encounter data missing.


# Proposed Solution
In this section, clearly describe a solution to the problem. The solution should be applicable to the project domain and appropriate for the dataset(s) or input(s) given. Provide enough detail (e.g., algorithmic description and/or theoretical properties) to convince us that your solution is applicable. Why might your solution work? Make sure to describe how the solution will be tested.

Given $x_1$ to $x_n$, each $\vec x_i$ $\in \mathbb{R}^d$ with features $f_1$ through $f_d$. Predict 
the most likely class $y_i$.

The solution is applicable and appropriate since:
- Given features and observations, a multi-class classification is appropriate for this dataset since our true music genre has different categorical values.
- We will build different models to produce the prediction, for example(not limited to) KNN, SVM, Neural Networks, Decision Trees, Gradient Boosting... The performance will be evaluated quantitatively.
- We will use (repeated) K-fold cross validation to split test data and test our model performace with accuracy if the data size is not enough.
- Comparision through models will be performed.

Model selection:
- We selected these models based on our multi-class classification(not limited to): LogisticRegression, RandomForest, Boosting, and Neural Network
- We will use LogisticRegression as a first baseline model because it's predictive power is relatively low but it is easier to understand, we will present a trial and get a basic understanding of multi-label classification. Also, this is our very beginning learning from the lecture, so we decided to give a start.
- RandomForest is a ensemble learning of decision trees, it is easy to understand and interpret. This will make our algorithm easier to visualize to give context to how it makes it's decision.
- Boosting is another ensemble learning but different with bagging(forest), it allows us to improve and boost the weak learner from previous and focus on more mis-classified data.
- Neural network is our exploration, NN is large and complicated. We want to implement a rough network within a medium size of neurons and present a solid output.

# Evaluation Metrics
For our project we have chosen to focus on the evaluation metrics of precision, recall, and f1 score which we will use to try to evaluate how well our project is predicting the music genre. Due to having to store our data in a measurable way through our data cleaning to be quantifiable we need to see if we are handling these new variables are correctly being measured in a way where important comparisons have the proper weights. Since we need the project to be measurable we need to see that these variables that we cleaned are being weighed correctly which is why we will use f1 to try to compare our prediction of our training set to the actual genre to try to measure how many true postives and true negatives we have in our predictions in order to see how accurate our predictions are. We will use precision to determine how good we are at classifying a sample positively (quality of positive predictions). We will use recall to determine how good we are at detecting positive samples.

We can also compare these scores to try to see if there is any overlap between predicting certain genres and see if our different models have more power in predicting these songs better than others depending on our score that we get. In general using these different factors that we have deemed quantifable we should check if we correctly measured the weight of these factors and see how truly we were able to identify the genre. 

# Results

#### Section 3,4,5 are main models.

If you went down any blind alleys that you later decided to not pursue, please don't abuse the TAs time by throwing in 81 lines of code and 4 plots related to something you actually abandoned.  Consider deleting things that are not important to your narrative.  If its slightly relevant to the narrative or you just want us to know you tried something, you could keep it in by summarizing the result in this report in a sentence or two, moving the actual analysis to another file in your repo, and providing us a link to that file.

### Subsection 1: Data and Clean
We want to predict the music genre given the attributes from the music. Our data is from Kaggle. We will try to implement 3 different models and compare their performance: Random Forest, AdaBoosting, Neural Network.

The dataset is relatively clean. There are only some missings in some random spots, which is completely blank, we interpret the reason as randomling missing or lack of information. So, we decide to drop all missing values.

Here, take a look at our data and the cleaning: [Data and Clean](https://nbviewer.org/github/COGS118A/Group021-Wi23/blob/main/FinalProjectCompleteVersion.ipynb#Data)

### Subsection 2: Features and Visualization

Firstly, we drop out the obtain date in the dataset since it is completely unrelated to our problem.(obtain date means the date when the collector receives the music data)

We have 17 features in total, but we decided to abandon some of them. Here is a quick introduciton:
- **Quantitaive** feature: popularity, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, mode, speechiness, tempo, valence, 12 features in total
- **Categorical** feature: key, mode, 2 features in total
- **Abandoned Unrelated** feature: instance_id, artist_name, track_name, 3 features in total

We categoize these 3 features as unrelated since they are just the names/composer names/ids.

Numerial features are always good to keep.

Categorcial features:

- **Key**: After we visualize the key and its music genre, we find that the genres are distributed evenly in every key. So, we believe that the feature key will not improve any of the models since it lacks of information and independence. In conclusion, we will not use key as one feature. [Here](https://nbviewer.org/github/COGS118A/Group021-Wi23/blob/main/FinalProjectCompleteVersion.ipynb#histogram) is our stacked histogram, **please** take a look, <span style="color:red">it is really **nice**!</span>.
- **Mode**: A feature only contains two possible values, easy to encode into 0 and 1, it has solid difference and independence, we will keep it.

Then, we plot the [correlation](https://nbviewer.org/github/COGS118A/Group021-Wi23/blob/main/FinalProjectCompleteVersion.ipynb#heatmap) and want to find out the max and min correlation, max:0.84, min:-0.79 which is acceptable, means that every feature is not strictly correlated. It is fine for us to keep every numerical feature.

#### Subsection 2+: Naive Tryout from Lecture TODO
Before the model, as the time after we are introduced with logistic regression, we are curious about its performance on a 10-label classification problem, so we did a navie tryout.

### Subsection 3: Random Forest TODO

Probably you need to describe the base model and demonstrate its performance.  Maybe you include a learning curve to show whether you have enough data to do train/validate/test split or have to go to k-folds or LOOCV or ???

### Subsection 4: AdaBoosting TODO

Perhaps some exploration of the model selection (hyper-parameters) or algorithm selection task. Validation curves, plots showing the variability of perfromance across folds of the cross-validation, etc. If you're doing one, the outcome of the null hypothesis test or parsimony principle check to show how you are selecting the best model.

### Subsection 5: Neural Network TODO

Maybe you do model selection again, but using a different kind of metric than before?


### Subsection 6: Outcomes TODO

Anything to say and guess from the accuracy

# Discussion

### Interpreting the result TODO, and COPY into complete version

OK, you've given us quite a bit of tech informaiton above, now its time to tell us what to pay attention to in all that.  Think clearly about your results, decide on one main point and 2-4 secondary points you want us to understand. Highlight HOW your results support those points.  You probably want 2-5 sentences per point.
 
- Main: It's difficult to reach a high accuracy in a 10-label classification. Most of the features are similar, not obvious enough, also ten genre, they are as clear as normal categories: planes/bikes...

- Since the data size is pretty large(40k+), Ensemble Learning which picks data in batches is our first thought. Explain randomF and boosting, and guess the reason why we reach limit

- Jerry Neural Network, hard to tune, also should try out different activation functions.

- Classify a music genre in general cannot be accomplished by machine, people's opnions,..., many ...

### Limitations TODO, and COPY into complete version

- Time and performance

- No efficient way to tune hyperparameters

- Tree is large, Neural Network is large, hard to visualize the model

- Not convincing, people have own standards to categorize genres, ...

### Ethics & Privacy
Due to the fact that each song labeled with artist and the name of the song there is the problem of these being identifable and since this data is likely not important to our analysis we will not reflect specific songs in our analysis and try to avoid biases due to song selections in our fold selection to the best of our ability. We will have our data highlight only the important aspects of the analysis that will not have identifiable links back to the data. We will have the songs instead represented as meaningless id code labels that will have no relation to the song or artist themselves to avoid potential bias in our data analysis and will instead focus on the other factors about the songs to focus instead on predicting the genre based on those quantifiable aspects rather than inherit biases we may unintentionally have if we based it on song or artist names. Due to the nature of us scrapping data that we ourselves did not collect we will comb through the data to make sure that there are no ethical issues with the data collection and that there is no bias to what was collected and that there is no PIID values that may have been collected in the data as discussed earlier. We will make sure that all genres are properly represented in valid ways and that our data analysis relys on purely numerical data that is comparable rather than our own bias. Furthermore if any ethical dilemas come up during or after our research we plan on adjusting our data to handle or remove ethical concerns within our modeling and deployment of the analysis. We will have our data in comparable and categorical matter where we weigh our data properly but if these factors are found to not be properly utilized or have a bias that affects our ethics we will try to reflect this in our final project.

Also, classifying music genre is a process about classifying music and making people more aware of music, which does not involve any extreme,  negative,  or positive evaluations. All output results, regardless of real facts, are objective from computer evaluation and will not negatively mislead society or the public. Further, we will also state that this scientific classification is only used as a reference in machine learning. When people are really willing to try to get in touch with music genre, they should consider everything comprehensively from both objective facts and subjective opinions. 

We all believe **Appreciation and Analysis of Music requires both rationality and sensibility**.

### Conclusion TODO, and COPY into complete version

After 3 model comparisons, we cannot over 60%...

# Footnotes
<a name="wikinote1"></a>1.[^](#wiki1): (16 February 2023) History of Music Wikipedia. https://en.wikipedia.org/wiki/History_of_music<br>
<a name="wikinote2"></a>2.[^](#wiki2): (9 February 2023) Music genre Wikipedia https://en.wikipedia.org/wiki/Music_genre#:~:text=A%20music%20genre%20or%20subgenre,a%20wide%20variety%20of%20subgenres<br>
<a name="newyorkernote"></a>3.[^](#newyorker): (15 March 2021) Genre Is Disappearing. What Comes Next? https://www.newyorker.com/magazine/2021/03/15/genre-is-disappearing-what-comes-next